In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from torch_geometric.data import Data
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

# ------------------------------------
# Configuration & Hyperparameters
# ------------------------------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Seeds for reproducibility
SEEDS = [8, 35, 43, 58, 70]

# GraphSAGE & training hyperparameters
SAGE_HIDDEN_DIM = 128
SAGE_DROPOUT = 0.2
LR = 4e-3
WEIGHT_DECAY = 1e-4
EPOCHS = 15000
PATIENCE = 1000

# ------------------------------------
# GNN Model Definition
# ------------------------------------
class GraphSAGENet(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.conv3 = SAGEConv(hidden_channels, hidden_channels)
        self.conv4 = SAGEConv(hidden_channels, hidden_channels)
        self.out = SAGEConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=SAGE_DROPOUT, training=self.training)
        x = F.relu(self.conv2(x, edge_index))
        x = F.dropout(x, p=SAGE_DROPOUT, training=self.training)
        x = F.relu(self.conv3(x, edge_index))
        x = F.dropout(x, p=SAGE_DROPOUT, training=self.training)
        x = F.relu(self.conv4(x, edge_index))
        x = F.dropout(x, p=SAGE_DROPOUT, training=self.training)
        return self.out(x, edge_index)

# ------------------------------------
# Train and Evaluate for One Seed
# ------------------------------------
def run_single_seed(X_raw, Y_raw, seed_model):
    # Set random seeds
    np.random.seed(seed_model)
    torch.manual_seed(seed_model)
    if DEVICE.type == 'cuda':
        torch.cuda.manual_seed_all(seed_model)

    # Preprocess features
    scaler = StandardScaler()
    X_np = scaler.fit_transform(np.abs(X_raw.cpu().numpy()))
    X = torch.tensor(X_np, dtype=torch.float32)

    # Create identity graph: only self-loops
    N = X.size(0)
    node_ids = torch.arange(N, device=DEVICE)
    edge_index = torch.stack([node_ids, node_ids], dim=0)  # shape [2, N]

    # Split indices
    idx = np.arange(N)
    train_idx, temp_idx = train_test_split(idx, test_size=0.2, random_state=seed_model)
    val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, random_state=seed_model)

    def get_mask(idxs):
        mask = torch.zeros(N, dtype=torch.bool)
        mask[idxs] = True
        return mask.to(DEVICE)

    data = Data(
        x=X.to(DEVICE),
        y=Y_raw.to(DEVICE),
        edge_index=edge_index
    )
    masks = {
        'train': get_mask(train_idx),
        'val': get_mask(val_idx),
        'test': get_mask(test_idx)
    }

    # Initialize model, optimizer, scheduler, and loss
    model = GraphSAGENet(X.size(1), SAGE_HIDDEN_DIM, Y_raw.size(1)).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=10)
    criterion = torch.nn.MSELoss()

    best_val = float('inf')
    patience_counter = 0

    # Training loop
    for epoch in range(EPOCHS):
        model.train()
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = criterion(out[masks['train']], data.y[masks['train']])
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            val_loss = criterion(model(data.x, data.edge_index)[masks['val']], data.y[masks['val']])
        scheduler.step(val_loss)

        if val_loss < best_val:
            best_val = val_loss
            best_state = {k: v.cpu() for k, v in model.state_dict().items()}
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE:
                break

    # Load best model
    model.load_state_dict(best_state)
    model.eval()
    with torch.no_grad():
        predictions = model(data.x, data.edge_index)

    # Compute metrics
    y_true = data.y
    r2 = r2_score(y_true[masks['test']].cpu().numpy(), predictions[masks['test']].cpu().numpy())
    mse = mean_squared_error(y_true[masks['test']].cpu().numpy(), predictions[masks['test']].cpu().numpy())

    return r2, mse

# ------------------------------------
# Run Experiment for Multiple Seeds
# ------------------------------------
def run_experiment_for_multiple_seeds(X_raw, Y_raw):
    r2_scores, mse_scores = [], []
    for seed in SEEDS:
        print(f"Running experiment with seed: {seed}")
        r2, mse = run_single_seed(X_raw, Y_raw, seed_model=seed)
        r2_scores.append(r2)
        mse_scores.append(mse)
        print(f"R²: {r2:.4f}, MSE: {mse:.4f}")

    print("\nResults across seeds:")
    print(f"Mean R²: {np.mean(r2_scores):.4f}, Std R²: {np.std(r2_scores):.4f}")
    print(f"Mean MSE: {np.mean(mse_scores):.4f}, Std MSE: {np.std(mse_scores):.4f}")

# ------------------------------------
# Main Execution
# ------------------------------------
if __name__ == "__main__":
    df = pd.read_csv("1000_spherical_anomalies.csv")
    X_raw = torch.tensor(df.iloc[:, :22].values, dtype=torch.float32)
    Y_raw = torch.tensor(df.iloc[:, 22:23].values, dtype=torch.float32)

    run_experiment_for_multiple_seeds(X_raw, Y_raw)

Running experiment with seed: 8
R²: 0.8443, MSE: 0.5020
Running experiment with seed: 35
R²: 0.8656, MSE: 0.4345
Running experiment with seed: 43
R²: 0.8617, MSE: 0.5628
Running experiment with seed: 58
R²: 0.8618, MSE: 0.4868
Running experiment with seed: 70
R²: 0.8512, MSE: 0.5108

Results across seeds:
Mean R²: 0.8569, Std R²: 0.0079
Mean MSE: 0.4994, Std MSE: 0.0413
